# ***Building Energy Benchmarking : Régression multilinéaire***

On va essayer d'entrainer un modèle de régression multilinéaire pour prédire la consomation et la pollution des immeubles via leurs caractéristiques. 
On entrainera les modèles avec la variable de l'EnergyStarScore et sans cette variable afin de voir si elle est utile pour les prédictions.

On cherche donc a obtenir les coéfficients $\beta_0, \beta_1, \cdots, \beta_n$ tels que la consomation d'une propriété $y$ puisse être calculée à l'aide de ses caractéristiques (numériques) $x_1, \cdots, x_n $ par :
$$ y = \beta_0 + \beta_1 x_1 + \cdots + \beta_n x_n$$

<h1>Table des matières<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Building-Energy-Benchmarking-:-Régression-multilinéaire" data-toc-modified-id="Building-Energy-Benchmarking-:-Régression-multilinéaire-1"><strong><em>Building Energy Benchmarking : Régression multilinéaire</em></strong></a></span></li><li><span><a href="#Partie-1-:-Préparation-des-données" data-toc-modified-id="Partie-1-:-Préparation-des-données-2"><strong>Partie 1 : Préparation des données</strong></a></span></li><li><span><a href="#Partie-2-:-Régression-sans-ENERGYSTAR-Score" data-toc-modified-id="Partie-2-:-Régression-sans-ENERGYSTAR-Score-3"><strong>Partie 2 : Régression sans ENERGYSTAR Score</strong></a></span></li><li><span><a href="#Partie-3-:-Régression-avec-ENERGYSTARScore" data-toc-modified-id="Partie-3-:-Régression-avec-ENERGYSTARScore-4"><strong>Partie 3 : Régression avec ENERGYSTARScore</strong></a></span></li></ul></div>

In [119]:
import numpy as np ; print("numpy version :", np.__version__)
import pandas as pd ; print("pandas version :",pd.__version__)
import sklearn as sk ; print("sklearn version :",sk.__version__)

numpy version : 1.23.2
matplotlib version : 3.5.3
seaborn version : 0.11.2
pandas version : 1.4.4
scipy version : 1.9.1
sklearn version : 1.1.2


# **Partie 1 : Préparation des données**

In [120]:
df = pd.read_csv("2016_Building_Energy_Benchmarking2.csv", sep=',', index_col=0)
df.dtypes

BuildingType                        object
ZipCode                            float64
Age                                  int64
NumberofBuildings                  float64
NumberofFloors                       int64
PropertyGFAParking                   int64
PropertyGFABuilding(s)               int64
LargestPropertyUseType              object
LargestPropertyUseTypeGFA          float64
SecondLargestPropertyUseType        object
SecondLargestPropertyUseTypeGFA    float64
ThirdLargestPropertyUseType         object
ThirdLargestPropertyUseTypeGFA     float64
ENERGYSTARScore                    float64
SiteEnergyUse(kBtu)                float64
TotalGHGEmissions                  float64
dtype: object

Je vais construire un dataframe de sorte à ce que ses features soient les suivantes :
*   Les différentes types de bâtiments dans lesquels on renseignera la surface totale du bâtiment (dans la colonne qui lui correspond)
*   Le nombre de bâtiments
*   Le nombre d'étages
*   L'age du bâtiment
*   L'aire du parking
*   Les différents types d'usage du bâtiment dans lesquels on renseigenra la surface utilisée pour chaque type d'usage
*   L'ENERGYSTAR Score
*   Le SiteEnergyUse(kBtu)
*   TotalGHGEmissions

Je met de côté le code postal car il n'y a pas de tedance linéaire sur la position.

In [121]:
#liste des features
features = ['Age', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFAParking']
features += df['BuildingType'].unique().tolist()
uses = [x for x in df['LargestPropertyUseType'].unique().tolist() if not(pd.isnull(x))==True]
uses += [x for x in df['SecondLargestPropertyUseType'].unique().tolist() if not(pd.isnull(x))==True]
uses += [x for x in df['ThirdLargestPropertyUseType'].unique().tolist() if not(pd.isnull(x))==True]
uses = set(uses)
features += uses
features += ['ENERGYSTARScore', 'SiteEnergyUse(kBtu)', 'TotalGHGEmissions']

# création du data frame avec les features
data = pd.DataFrame(columns=features)

# copie des colonnes identiques 
data['Age'] = df['Age']
data['NumberofBuildings'] = df['NumberofBuildings']
data['NumberofFloors'] = df['NumberofFloors']
data['PropertyGFAParking'] = df['PropertyGFAParking']
data['ENERGYSTARScore'] = df['ENERGYSTARScore']
data['SiteEnergyUse(kBtu)'] = df['SiteEnergyUse(kBtu)']
data['TotalGHGEmissions'] = df['TotalGHGEmissions']

# surfaces total du bâtiment en fonction de leur type
for btype in data.columns[4:12]:
    idx = df.loc[df['BuildingType']==btype].index
    data.loc[idx, btype] = df.iloc[idx]['PropertyGFABuilding(s)']

# surfaces utilisées en fonction de l'usage
for use in data.columns[12:-3]:
    idx = df.loc[df['LargestPropertyUseType']==use].index
    data.loc[idx, use] = df.iloc[idx]['LargestPropertyUseTypeGFA']
for use in data.columns[12:-3]:
    idx = df.loc[df['SecondLargestPropertyUseType']==use].index
    data.loc[idx, use] = df.iloc[idx]['SecondLargestPropertyUseTypeGFA']
for use in data.columns[12:-3]:
    idx = df.loc[df['ThirdLargestPropertyUseType']==use].index
    data.loc[idx, use] = df.iloc[idx]['ThirdLargestPropertyUseTypeGFA']

# remplissage des nan par 0 pour les nan sur les sur colonnes de surfaces
data[data.columns[4:-3]] = data[data.columns[4:-3]].fillna(0)

# remplissage des nan par 1 sur NumberofBuildings
data['NumberofBuildings'] = data['NumberofBuildings'].fillna(1)

# convertion du data type des colonnes en float
data = data.astype(float)

data.head()

,Age,NumberofBuildings,NumberofFloors,PropertyGFAParking,NonResidential,Nonresidential COS,Multifamily MR (5-9),SPS-District K-12,Campus,Multifamily LR (1-4),...,Hotel,Courthouse,Strip Mall,Performing Arts,Convenience Store without Gas Station,Bank Branch,Financial Office,ENERGYSTARScore,SiteEnergyUse(kBtu),TotalGHGEmissions
0,95.0,1.0,12.0,0.0,88434.0,0.0,0.0,0.0,0.0,0.0,...,88434.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,7226362.5,249.98
1,26.0,1.0,11.0,15064.0,88502.0,0.0,0.0,0.0,0.0,0.0,...,83880.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,8387933.0,295.86
2,53.0,1.0,41.0,196718.0,759392.0,0.0,0.0,0.0,0.0,0.0,...,756493.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,72587024.0,2089.28
3,96.0,1.0,10.0,0.0,61320.0,0.0,0.0,0.0,0.0,0.0,...,61320.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,6794584.0,286.43
4,42.0,1.0,18.0,62000.0,113580.0,0.0,0.0,0.0,0.0,0.0,...,123445.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0,14172606.0,505.01


On remplira les NaN sur les valeurs à prédire par leur médiane (la régréssion ne gère pas les NaN sur l'entrainement).
On appliquera dans un premier temps un recentrage des données pour mieux interpréter l'ordonnée à l'origine $\beta_0 $ : elle correspondra ainsi à ce que l'on obtiendrai pour une propriété qui aurait la moyenne sur chaque feature.
Puis on normalise les données à l'aide de l'écart-type.

In [122]:
# recentrage des données autour de la moyenne
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
data[data.columns]= std.fit_transform(data[data.columns])

# NaN des features à prédire
data['SiteEnergyUse(kBtu)'] = data['SiteEnergyUse(kBtu)'].fillna(data['SiteEnergyUse(kBtu)'].median())
data['TotalGHGEmissions'] = data['TotalGHGEmissions'].fillna(data['TotalGHGEmissions'].median())

Dans la suite on implémente une régression multilinéaire pour chacun des deux cas. Pour cela on séparera les données en un set d'entrainement et un set de test. On utilisera le R2 Score, le Mean Square Error et le Mean Absolute Error pour mesurer l'efficacité de la régression. On entrainera la régression à l'aide d'une validation croisée afin d'affiner les paramètres et eviter l'overfitting sur les données.

In [123]:
data.head()

,Age,NumberofBuildings,NumberofFloors,PropertyGFAParking,NonResidential,Nonresidential COS,Multifamily MR (5-9),SPS-District K-12,Campus,Multifamily LR (1-4),...,Hotel,Courthouse,Strip Mall,Performing Arts,Convenience Store without Gas Station,Bank Branch,Financial Office,ENERGYSTARScore,SiteEnergyUse(kBtu),TotalGHGEmissions
0,1.256623,-0.050644,1.327146,-0.247557,0.376788,-0.075493,-0.325671,-0.135227,-0.028876,-0.480681,...,2.630990,-0.019519,-0.037163,-0.036723,-0.024022,-0.059479,-0.030855,-0.294725,0.084355,0.241774
1,-0.829025,-0.050644,1.145118,0.218504,0.377348,-0.075493,-0.325671,-0.135227,-0.028876,-0.480681,...,2.490284,-0.019519,-0.037163,-0.036723,-0.024022,-0.059479,-0.030855,-0.257506,0.138113,0.326933
2,-0.012902,-0.050644,6.605967,5.838650,5.906975,-0.075493,-0.325671,-0.135227,-0.028876,-0.480681,...,23.272252,-0.019519,-0.037163,-0.036723,-0.024022,-0.059479,-0.030855,-0.927449,3.109272,3.655773
3,1.286849,-0.050644,0.963089,-0.247557,0.153308,-0.075493,-0.325671,-0.135227,-0.028876,-0.480681,...,1.793239,-0.019519,-0.037163,-0.036723,-0.024022,-0.059479,-0.030855,-0.443602,0.064372,0.309430
4,-0.345396,-0.050644,2.419316,1.670645,0.584047,-0.075493,-0.325671,-0.135227,-0.028876,-0.480681,...,3.712738,-0.019519,-0.037163,-0.036723,-0.024022,-0.059479,-0.030855,0.263560,0.405830,0.715145


# **Partie 2 : Régression sans ENERGYSTAR Score**

On sélectionne les données à prédire et les données. 

In [124]:
x=data.iloc[:,:-3]
y1 = data['SiteEnergyUse(kBtu)']
y2 = data['TotalGHGEmissions']

On sépare les données en un set d'entrainement et un set de test.

In [125]:
from sklearn.model_selection import train_test_split

test_size=0.2
x1_train, x1_test, y1_train, y1_test = train_test_split(x, y1, test_size=test_size, random_state=2)
x2_train, x2_test, y2_train, y2_test = train_test_split(x, y2, test_size=test_size, random_state=2)

On entraine les deux modèles pour les deux variables à prédire à l'aide d'une validation croisée. 

In [126]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

folds = 20
hyper_params = {}

linear1 = LinearRegression()
cv1 = GridSearchCV(estimator = linear1, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1) 
cv1.fit(x1_train, y1_train)
y1_pred = cv1.predict(x1_test)

linear2 = LinearRegression()
cv2 = GridSearchCV(estimator = linear2, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1) 
cv2.fit(x2_train, y2_train)
y2_pred = cv2.predict(x2_test)

Fitting 20 folds for each of 1 candidates, totalling 20 fits
Fitting 20 folds for each of 1 candidates, totalling 20 fits


On affiche les résultats obtenus par les algorithmes sur le data set de test. 

In [128]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print('SiteEnergyUse(kBtu)')
print('r2_score : ', r2_score(y1_test, y1_pred))
print('mean_squared_error : ', mean_squared_error(y1_test, y1_pred))
print('mean_absolute_error : ', mean_absolute_error(y1_test, y1_pred))

print('\nTotalGHGEmissions')
print('r2_score : ', r2_score(y2_test, y2_pred))
print('mean_squared_error : ', mean_squared_error(y2_test, y2_pred))
print('mean_absolute_error : ', mean_absolute_error(y2_test, y2_pred))

SiteEnergyUse(kBtu)
r2_score :  0.6979357132869346
mean_squared_error :  0.07306042969942787
mean_absolute_error :  0.09847256278842961

TotalGHGEmissions
r2_score :  0.5291661328191066
mean_squared_error :  0.16957096589022075
mean_absolute_error :  0.1598221413965945


On remarque que l'algorithme n'explique pas 30% de la variance du problème, donnant donc des résultats aproximatifs.

# **Partie 3 : Régression avec ENERGYSTARScore**

On refait les mêmes manipulations que la partie précédente avec la feature ENERGYSTARScore en plus.

In [129]:
x=data.dropna(subset='ENERGYSTARScore')
y1 = x['SiteEnergyUse(kBtu)']
y2 = x['TotalGHGEmissions']
x=x.iloc[:,:-2]

In [130]:
from sklearn.model_selection import train_test_split

test_size=0.2
x1_train, x1_test, y1_train, y1_test = train_test_split(x, y1, test_size=test_size, random_state=1)
x2_train, x2_test, y2_train, y2_test = train_test_split(x, y2, test_size=test_size, random_state=1)

In [131]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

folds = 20
hyper_params = {}

linear1 = LinearRegression()
cv1 = GridSearchCV(estimator = linear1, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1) 
cv1.fit(x1_train, y1_train)
y1_pred = cv1.predict(x1_test)

linear2 = LinearRegression()
cv2 = GridSearchCV(estimator = linear2, param_grid = hyper_params, scoring= 'r2', cv = folds, verbose = 1) 
cv2.fit(x2_train, y2_train)
y2_pred = cv2.predict(x2_test)

Fitting 20 folds for each of 1 candidates, totalling 20 fits
Fitting 20 folds for each of 1 candidates, totalling 20 fits


In [133]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print('SiteEnergyUse(kBtu)')
print('r2_score : ', r2_score(y1_test, y1_pred))
print('mean_squared_error : ', mean_squared_error(y1_test, y1_pred))
print('mean_absolute_error : ', mean_absolute_error(y1_test, y1_pred))

print('\nTotalGHGEmissions')
print('r2_score : ', r2_score(y2_test, y2_pred))
print('mean_squared_error : ', mean_squared_error(y2_test, y2_pred))
print('mean_absolute_error : ', mean_absolute_error(y2_test, y2_pred))

SiteEnergyUse(kBtu)
r2_score :  0.8608084181065075
mean_squared_error :  0.12187752522115161
mean_absolute_error :  0.08288087823232863

TotalGHGEmissions
r2_score :  0.9663684823155643
mean_squared_error :  0.07126624429628092
mean_absolute_error :  0.10785377124752653


Avec la variable ENERGYSTARScore on obtient des performances très bonnes, dépassant même les 0.95 pour les emissions de polluants. 

Cette feature n'est pas écartable pour un modèle de régression multilinéaire.